# Source contributions

In [1]:
import pickle
import numpy as np
import matplotlib
import netCDF4 as nc

###### Parameters:

In [2]:
# domain dimensions:
imin, imax = 1480, 2180
jmin, jmax = 160, 800
isize = imax-imin
jsize = jmax-jmin

# model result set:
folder_ref      = '/data/brogalla/run_storage/Mn-reference-202103/' # Mn-reference experiment
folder_cleanice = '/data/brogalla/run_storage/Mn-clean-ice-202103/' # Mn-clean-ice
folder_spm      = '/data/brogalla/run_storage/Mn-spm-202103/'       # Mn-upper-bound-rivers

In [36]:
years = [2015] # [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, \
#         2017, 2018, 2019]

###### Load files:

In [11]:
mask      = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask     = np.array(mask.variables['tmask'])[0,:,imin:imax,jmin:jmax]
land_mask = np.ma.masked_where((tmask[:,:,:] > 0.1), tmask[:,:,:]) 
e1t_base  = np.array(mask.variables['e1t'])[0,imin:imax,jmin:jmax]
e2t_base  = np.array(mask.variables['e2t'])[0,imin:imax,jmin:jmax]
e3t       = np.array(mask.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]
e3t_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), e3t)

e1t       = np.tile(e1t_base, (50,1,1))
e2t       = np.tile(e2t_base, (50,1,1))
volume    = e1t*e2t*e3t
area_base = e1t_base*e2t_base
volume_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), volume)
area_masked   = np.ma.masked_where((tmask[0,:,:] < 0.1), area_base)

lons      = np.array(mask.variables['nav_lon'])
lats      = np.array(mask.variables['nav_lat'])

##### Functions:

In [12]:
def load_results(folder_ref, year, experiment):
    
    months  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    dmn_riv = np.empty((12,isize,jsize))   ; dmn_sed = np.empty((12,isize,jsize)); 
    dmn_atm = np.empty((12,isize,jsize))   ; dmn_ice = np.empty((12,isize,jsize));
    dmn_sedice = np.empty((12,isize,jsize));
    dmn_red = np.empty((12,50,isize,jsize)); dmn_oxi = np.empty((12,50,isize,jsize));
    dmn_priv = np.empty((12,isize,jsize))   ;
    
    for i, month in enumerate(months):
        file        = f'ANHA12_EXH006_1m_{year}0101_{year}1231_comp_{year}{month}-{year}{month}.nc'
        ref_monthly = nc.Dataset(folder_ref+file)
        
        dmn_riv[i,:,:]    = np.array(ref_monthly.variables['dmnriv'])[0,:,:]
        
        if experiment=='spm': # particle origin riverine Mn contribution included
            dmn_priv[i,:,:] = np.array(ref_monthly.variables['pmnriv'])[0,:,:]
        else: # no particle origin riverine Mn
            dmn_priv[i,:,:] = 0
        
        dmn_sed[i,:,:]    = np.array(ref_monthly.variables['dmnsed'])[0,:,:]
        dmn_sedice[i,:,:] = np.array(ref_monthly.variables['dmnsedice'])[0,:,:]
        dmn_atm[i,:,:]    = np.array(ref_monthly.variables['dmnatm'])[0,:,:]
        dmn_ice[i,:,:]    = np.array(ref_monthly.variables['dmnice'])[0,:,:]
        dmn_red[i,:,:,:]  = np.array(ref_monthly.variables['dmnred'])[0,:,:,:]
        dmn_oxi[i,:,:,:]  = np.array(ref_monthly.variables['dmnoxi'])[0,:,:,:]
    
    tmask_surf    = np.empty(dmn_riv.shape)
    tmask_surf[:] = tmask[0,:,:]
    tmask_full    = np.empty(dmn_red.shape)
    tmask_full[:] = tmask
    
    # Mask points on land:
    dmn_mriv    = np.ma.masked_where((tmask_surf < 0.1), dmn_riv); 
    dmn_mpriv   = np.ma.masked_where((tmask_surf < 0.1), dmn_priv); 
    dmn_msed    = np.ma.masked_where((tmask_surf < 0.1), dmn_sed)
    dmn_msedice = np.ma.masked_where((tmask_surf < 0.1), dmn_sedice)
    dmn_matm    = np.ma.masked_where((tmask_surf < 0.1), dmn_atm)
    dmn_mice    = np.ma.masked_where((tmask_surf < 0.1), dmn_ice)
    dmn_mred    = np.ma.masked_where((tmask_full < 0.1), dmn_red)
    dmn_moxi    = np.ma.masked_where((tmask_full < 0.1), dmn_oxi)

    return dmn_mriv, dmn_mpriv, dmn_msed, dmn_msedice, dmn_matm, dmn_mice, dmn_mred , dmn_moxi

In [13]:
def calculate_contribution(year, mask, experiment='ref'):
    # Calculate the contribution of the model components by region:
    
    if experiment=='spm':
        folder_year = folder_spm + f'ref-{year}-spm-202103/'
    else:
        folder_year = folder_ref + f'ref-{year}-202103/'
    
    dmn_mriv, dmn_mpriv, dmn_msed, dmn_msedice, dmn_matm, dmn_mice, dmn_mred, dmn_moxi = \
                    load_results(folder_year, year, experiment)
    
    # Calculate contributions to the upper water column, so mask locations where sediment resuspension is added in the deep.
    indexes_bottom = np.tile(index_bottom, (12,1,1))
    dmn_msed = np.ma.masked_where(indexes_bottom > 19, dmn_msed)
 

    # moles per second
    priv   = np.ma.masked_where(mask==0, dmn_mpriv)   * volume_masked[0,:,:] * 1e3
    riv    = np.ma.masked_where(mask==0, dmn_mriv)    * volume_masked[0,:,:] * 1e3
    sed    = np.ma.masked_where(mask==0, dmn_msed)    * volume_bottom * 1e3
    sedice = np.ma.masked_where(mask==0, dmn_msedice) * volume_masked[0,:,:] * 1e3
    atm    = np.ma.masked_where(mask==0, dmn_matm)    * volume_masked[0,:,:] * 1e3
    ice    = np.ma.masked_where(mask==0, dmn_mice)    * volume_masked[0,:,:] * 1e3

    # May and June contribution to domain: moles / 2 months
    atm_year    = np.ma.sum(atm[4:6,:,:], axis=(0,1,2))   *3600*24*(31+30)
    riv_year    = np.ma.sum(riv[4:6,:,:], axis=(0,1,2))   *3600*24*(31+30)
    priv_year   = np.ma.sum(priv[4:6,:,:], axis=(0,1,2))  *3600*24*(31+30)
    sed_year    = np.ma.sum(sed[4:6,:,:], axis=(0,1,2))   *3600*24*(31+30)
    sedice_year = np.ma.sum(sedice[4:6,:,:], axis=(0,1,2))*3600*24*(31+30)
    ice_year    = np.ma.sum(ice[4:6,:,:], axis=(0,1,2))   *3600*24*(31+30)
  
    
#     # Yearly contribution to domain: moles / year
#     atm_year    = np.ma.sum(atm[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
#     riv_year    = np.ma.sum(riv[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
#     priv_year   = np.ma.sum(priv[:,:,:], axis=(0,1,2))  *3600*24*365 / 12
#     sed_year    = np.ma.sum(sed[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
#     sedice_year = np.ma.sum(sedice[:,:,:], axis=(0,1,2))*3600*24*365 / 12
#     ice_year    = np.ma.sum(ice[:,:,:], axis=(0,1,2))   *3600*24*365 / 12
    
    # Average yearly contribution over domain: moles / m2 / year
    total_area     = np.ma.sum(np.ma.masked_where(mask[0,:,:]==0, area_masked[:,:]))
    atmm2_year     =  atm_year    / total_area
    rivm2_year     =  riv_year    / total_area
    privm2_year    =  priv_year   / total_area
    sedm2_year     =  sed_year    / total_area 
    sedicem2_year  =  sedice_year / total_area
    icem2_year     =  ice_year    / total_area                                   
    
    total_yearly_contribution = np.array([priv_year, riv_year, sed_year, sedice_year, ice_year, atm_year])  # mol/yr
    ave_yearly_contribution   = np.array([privm2_year, rivm2_year, sedm2_year, sedicem2_year, icem2_year, \
                                          atmm2_year]) # mol/m2/yr
    
    return total_yearly_contribution, ave_yearly_contribution

In [14]:
def calculate_regional_contributions(mask):
    
    totals_ref   = np.empty((len(years),6)); totals_spm   = np.empty((len(years),6))
    averages_ref = np.empty((len(years),6)); averages_spm = np.empty((len(years),6))

    totals_ref[:]   = np.NaN; totals_spm[:]    = np.NaN;
    averages_ref[:] = np.NaN; averages_spm[:]  = np.NaN;

    for i, year in enumerate(years):
        total_ref_year, ave_ref_year           = calculate_contribution(year, mask, experiment='ref')
        total_spm_year, ave_spm_year           = calculate_contribution(year, mask, experiment='spm')

        totals_ref[i,:]        = total_ref_year 
        totals_spm[i,:]        = total_spm_year 
        averages_ref[i,:]      = ave_ref_year  
        averages_spm[i,:]      = ave_spm_year 
    
    return totals_ref, totals_spm, averages_ref, averages_spm

In [15]:
def reshape_arrays(array_in):
    fixed      = np.zeros((len(years),5))
    
    for i in range(0,5):
        if i==0: # Sum the particulate and dissolved river contributions
            fixed[:,i] = array_in[:,i]+array_in[:,i+1]
        else:
            fixed[:,i] = array_in[:,i+1]
    
    return fixed

In [16]:
def pipeline(totals_input_ref, totals_input_spm, averages_input_ref, averages_input_spm):
    
    regions = ['CB', 'CAA', 'wCAA', 'eCAA']
    
    totals_fixed_ref = totals_input_ref.copy(); averages_fixed_ref = averages_input_ref.copy();
    totals_fixed_spm = totals_input_spm.copy(); averages_fixed_spm = averages_input_spm.copy();
    
    # Reshape arrays so that particulate and dissolved river contributions are combined into one.
    for region in regions:

        totals_fixed_ref[region]   = reshape_arrays(totals_input_ref[region])
        totals_fixed_spm[region]   = reshape_arrays(totals_input_spm[region])
        averages_fixed_ref[region] = reshape_arrays(averages_input_ref[region])
        averages_fixed_spm[region] = reshape_arrays(averages_input_spm[region])
    
    return totals_fixed_ref, totals_fixed_spm, averages_fixed_ref, averages_fixed_spm

In [17]:
def print_interannual_averages(totals_ref, totals_spm, averages_ref, averages_spm, location='Full'):

    print('Interannual average values in ' + location)
    print('----------------------------------------------------------------')
    total_ref_annual = np.sum(totals_ref[location], axis=1)
    total_spm_annual = np.sum(totals_spm[location], axis=1)
    
    print(f"{'Total annual contribution of Mn [Mmol/yr]:':<55}" +
          f"{np.average(total_ref_annual)*1e-6:<5.0f} ({np.average(total_spm_annual)*1e-6:<5.0f})")
    print(f"{'-- River discharge ---':<40}" +
          f" {np.average(totals_ref[location][:,0])*1e-6:<8.2f} ({np.average(totals_spm[location][:,0])*1e-6:<8.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f" {np.average(totals_ref[location][:,1])*1e-6:<8.2f} ({np.average(totals_spm[location][:,1])*1e-6:<8.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f" {np.average(totals_ref[location][:,2])*1e-6:<8.2f} ({np.average(totals_spm[location][:,2])*1e-6:<8.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f" {np.average(totals_ref[location][:,3])*1e-6:<8.2f} ({np.average(totals_spm[location][:,3])*1e-6:<8.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f" {np.average(totals_ref[location][:,4])*1e-6:<8.2f} ({np.average(totals_spm[location][:,4])*1e-6:<8.2f})") 

    tref = np.sum(totals_ref[location], axis=(0,1))
    tspm = np.sum(totals_spm[location], axis=(0,1))

    print('----------------------------------------------------------------')
    print('Total annual contribution of Mn [%]:') 
    print(f"{'-- River discharge ---':<40}" +
          f"{np.sum(totals_ref[location][:,0])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,0])*100/tspm:<5.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f"{np.sum(totals_ref[location][:,1])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,1])*100/tspm:<5.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f"{np.sum(totals_ref[location][:,2])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,2])*100/tspm:<5.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f"{np.sum(totals_ref[location][:,3])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,3])*100/tspm:<5.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f"{np.sum(totals_ref[location][:,4])*100/tref:<5.2f} ({np.sum(totals_spm[location][:,4])*100/tspm:<5.2f})")

    print('----------------------------------------------------------------')
    print(f"{'Average annual contribution of Mn [micromol/m2/yr]:':<55}" +
          f"{np.average(np.sum(averages_ref[location], axis=1)*1e6):<5.0f}" +
          f"({np.average(np.sum(averages_spm[location], axis=1)*1e6):<5.0f})")
    print(f"{'-- River discharge ---':<40}" +
          f"{np.average(averages_ref[location][:,0])*1e6:<8.2f} ({np.average(averages_spm[location][:,0])*1e6:<8.2f})")
    print(f"{'-- Sediment resuspension ---':<40}" +
          f"{np.average(averages_ref[location][:,1])*1e6:<8.2f} ({np.average(averages_spm[location][:,1])*1e6:<8.2f})")
    print(f"{'-- Sediment released by sea ice ---':<40}" +
          f"{np.average(averages_ref[location][:,2])*1e6:<8.2f} ({np.average(averages_spm[location][:,2])*1e6:<8.2f})")
    print(f"{'-- Dust released by sea ice ---':<40}" +
          f"{np.average(averages_ref[location][:,3])*1e6:<8.2f} ({np.average(averages_spm[location][:,3])*1e6:<8.2f})")
    print(f"{'-- Dust deposition ---':<40}" +
          f"{np.average(averages_ref[location][:,4])*1e6:<8.2f} ({np.average(averages_spm[location][:,4])*1e6:<8.2f})")

    return

### Calculations:

Find grid cell volume and thickness at ocean floor:

In [18]:
tmask_bot = np.copy(tmask)
tmask_bot[0,:,:] = 1
e3t_mask = np.ma.masked_where((tmask_bot[:,:,:] < 0.1), e3t[:,:,:])

# find bottom grid cell index:
ind = np.array(np.ma.notmasked_edges(e3t_mask[:,:,:], axis=0))

# array of shape (isize, jsize) containing bottom grid cell index
index_bottom = np.ma.masked_where((tmask[0,:,:] < 0.1), ind[1][0][:].reshape(isize,jsize))
# index_bottom = np.ma.argmax(e3t_masked[:,:,:], axis=0)

In [19]:
e3t_bot = np.zeros((isize,jsize))
vol_bot = np.zeros((isize,jsize))

for i in range(0, isize):
    for j in range(0,jsize):
        k = index_bottom[i,j]
        try:
            e3t_bot[i,j] = e3t[k,i,j]
            vol_bot[i,j] = volume[k,i,j]
        except:
            e3t_bot[i,j] = np.nan
            vol_bot[i,j] = np.nan
        
e3t_bottom    = np.ma.masked_where((tmask[0,:,:] < 0.1), np.ma.masked_where(np.isnan(e3t_bot), e3t_bot))
volume_bottom = np.ma.masked_where((tmask[0,:,:] < 0.1), np.ma.masked_where(np.isnan(vol_bot), vol_bot))

Create masks for specific subregions:
- Canada Basin
- CAA
- Full domain
- Western CAA (separated by Barrow Sill)
- Eastern CAA (separated by Barrow Sill)

In [20]:
# Find indices to separate out Canada Basin:

x_ind = np.arange(1620, 2100, 1)
y_ind = (-7/8)*x_ind + 1517 + 700

CB_indx = []
CB_indy = []

for index in range(0,len(x_ind)):
    CB_x = np.arange(x_ind[index],2180,1)
    CB_y = np.ones(CB_x.shape)*y_ind[index]
    
    CB_indx = np.append(CB_x, CB_indx)
    CB_indy = np.append(CB_y, CB_indy)

In [21]:
# Separate Canada Basin and the CAA:
mask_ini_CB  = np.zeros((isize,jsize))
mask_ini_CAA = np.ones((isize,jsize))

for i, j in zip(CB_indx, CB_indy):
    mask_ini_CB[int(i-imin),int(j-jmin)]  = 1
    mask_ini_CAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_CB[150:-1 ,-8:-1] = 1
mask_ini_CAA[150:-1,-8:-1] = 0


In [22]:
mask_ini_wCAA = np.zeros((isize,jsize))
mask_ini_eCAA = np.zeros((isize,jsize))

# Separate the western and eastern CAA:
mask_ini_wCAA = np.where(lons[imin:imax,jmin:jmax] < -100, 1, 0)
mask_ini_eCAA = np.where(lons[imin:imax,jmin:jmax] > -100, 1, 0)

for i, j in zip(CB_indx, CB_indy):
    mask_ini_wCAA[int(i-imin),int(j-jmin)] = 0
    mask_ini_eCAA[int(i-imin),int(j-jmin)] = 0
    
mask_ini_wCAA[150:-1,-8:-1] = 0
mask_ini_eCAA[150:-1,-8:-1] = 0

In [23]:
mask_CB_yr    = np.tile(mask_ini_CB  , (12,1,1))
mask_CAA_yr   = np.tile(mask_ini_CAA , (12,1,1))
mask_full_yr  = np.ones((12, isize, jsize))
mask_wCAA_yr  = np.tile(mask_ini_wCAA, (12,1,1))
mask_eCAA_yr  = np.tile(mask_ini_eCAA, (12,1,1))

In [24]:
mask_CB_depth = np.tile(mask_CB_yr  , (50,1,1,1))
mask_CB_depth = mask_CB_depth.reshape(12,50,700,640)

### Calculate:

In [29]:
CB_totals_ref, CB_totals_spm, CB_averages_ref, CB_averages_spm = calculate_regional_contributions(mask_CB_yr)

In [30]:
CAA_totals_ref, CAA_totals_spm, CAA_averages_ref, CAA_averages_spm = calculate_regional_contributions(mask_CAA_yr)

In [31]:
wCAA_totals_ref, wCAA_totals_spm, wCAA_averages_ref, wCAA_averages_spm = calculate_regional_contributions(mask_wCAA_yr)

In [32]:
eCAA_totals_ref, eCAA_totals_spm, eCAA_averages_ref, eCAA_averages_spm = calculate_regional_contributions(mask_eCAA_yr)

### Pickle the results from these calculations:

In [ ]:
pickle.dump((CB_totals_ref, CAA_totals_ref, wCAA_totals_ref, eCAA_totals_ref),\
            open('calculations/ref-total-June2015_component_contributions.pickle','wb'))

In [ ]:
pickle.dump((CB_totals_spm, CAA_totals_spm, wCAA_totals_spm, eCAA_totals_spm),\
            open('calculations/spm-total-June2015_component_contributions.pickle','wb'))

In [ ]:
pickle.dump((CB_averages_ref, CAA_averages_ref, wCAA_averages_ref, eCAA_averages_ref),\
            open('calculations/ref-ave-June2015_component_contributions.pickle','wb'))

In [ ]:
pickle.dump((CB_averages_spm, CAA_averages_spm, wCAA_averages_spm, eCAA_averages_spm),\
            open('calculations/spm-ave-June2015_component_contributions.pickle','wb'))

### Overview:

In [ ]:
folder = '/ocean/brogalla/GEOTRACES/analysis-brogalla/paper2/calculations'
CB_totals_ref, CAA_totals_ref, wCAA_totals_ref, eCAA_totals_ref = \
                pickle.load(open(f'{folder}/ref-total-June2015_component_contributions.pickle','rb')) 
CB_totals_spm, CAA_totals_spm, wCAA_totals_spm, eCAA_totals_spm = \
                pickle.load(open(f'{folder}/spm-total-June2015_component_contributions.pickle','rb')) 
CB_averages_ref, CAA_averages_ref, wCAA_averages_ref, eCAA_averages_ref = \
                pickle.load(open(f'{folder}/ref-ave-June2015_component_contributions.pickle','rb')) 
CB_averages_spm, CAA_averages_spm, wCAA_averages_spm, eCAA_averages_spm = \
                pickle.load(open(f'{folder}/spm-ave-June2015_component_contributions.pickle','rb')) 

In [33]:
folder = '/ocean/brogalla/GEOTRACES/analysis-brogalla/paper1/calculations/'
CB_totals_ref, CAA_totals_ref, wCAA_totals_ref, eCAA_totals_ref = \
                pickle.load(open(f'{folder}/ref-total_component_contributions-20210418.pickle','rb')) 
CB_totals_spm, CAA_totals_spm, wCAA_totals_spm, eCAA_totals_spm = \
                pickle.load(open(f'{folder}/spm-total_component_contributions-20210418.pickle','rb')) 
CB_averages_ref, CAA_averages_ref, wCAA_averages_ref, eCAA_averages_ref = \
                pickle.load(open(f'{folder}/ref-averages_component_contributions-20210418.pickle','rb')) 
CB_averages_spm, CAA_averages_spm, wCAA_averages_spm, eCAA_averages_spm = \
                pickle.load(open(f'{folder}/spm-averages_component_contributions-20210418.pickle','rb')) 

In [34]:
ref_totals   = {'CB': CB_totals_ref, 'CAA':CAA_totals_ref, 'wCAA':wCAA_totals_ref, 'eCAA':eCAA_totals_ref}
spm_totals   = {'CB': CB_totals_spm, 'CAA':CAA_totals_spm, 'wCAA':wCAA_totals_spm, 'eCAA':eCAA_totals_spm}
ref_averages = {'CB': CB_averages_ref, 'CAA':CAA_averages_ref, 'wCAA':wCAA_averages_ref, 'eCAA':eCAA_averages_ref}
spm_averages = {'CB': CB_averages_spm, 'CAA':CAA_averages_spm, 'wCAA':wCAA_averages_spm, 'eCAA':eCAA_averages_spm}

In [37]:
ref_totals   = {'CB': CB_totals_ref[14:15,:], 'CAA':CAA_totals_ref[14:15,:], 'wCAA':wCAA_totals_ref[14:15,:], 'eCAA':eCAA_totals_ref[14:15,:]}
spm_totals   = {'CB': CB_totals_spm[14:15,:], 'CAA':CAA_totals_spm[14:15,:], 'wCAA':wCAA_totals_spm[14:15,:], 'eCAA':eCAA_totals_spm[14:15,:]}
ref_averages = {'CB': CB_averages_ref[14:15,:], 'CAA':CAA_averages_ref[14:15,:], 'wCAA':wCAA_averages_ref[14:15,:], 'eCAA':eCAA_averages_ref[14:15,:]}
spm_averages = {'CB': CB_averages_spm[14:15,:], 'CAA':CAA_averages_spm[14:15,:], 'wCAA':wCAA_averages_spm[14:15,:], 'eCAA':eCAA_averages_spm[14:15,:]}

In [38]:
final_ref_totals, final_spm_totals, \
    final_ref_averages, final_spm_averages = pipeline(ref_totals, spm_totals, ref_averages, spm_averages)

In [107]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CB')

Interannual average values in CB
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             164   (182  )
-- River discharge ---                   5.75     (23.79   )
-- Sediment resuspension ---             14.14    (14.14   )
-- Sediment released by sea ice ---      144.19   (144.19  )
-- Dust released by sea ice ---          0.02     (0.02    )
-- Dust deposition ---                   0.04     (0.04    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  3.50  (13.06)
-- Sediment resuspension ---            8.61  (7.76 )
-- Sediment released by sea ice ---     87.85 (79.15)
-- Dust released by sea ice ---         0.01  (0.01 )
-- Dust deposition ---                  0.02  (0.02 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    82   (91   )
-- River discharge --- 

In [39]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CB')

Interannual average values in CB
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             383   (414  )
-- River discharge ---                   10.20    (41.41   )
-- Sediment resuspension ---             42.30    (42.30   )
-- Sediment released by sea ice ---      330.07   (330.08  )
-- Dust released by sea ice ---          0.34     (0.34    )
-- Dust deposition ---                   0.03     (0.03    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  2.66  (10.00)
-- Sediment resuspension ---            11.05 (10.21)
-- Sediment released by sea ice ---     86.20 (79.70)
-- Dust released by sea ice ---         0.09  (0.08 )
-- Dust deposition ---                  0.01  (0.01 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    717  (732  )
-- River discharge --- 

In [108]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CAA')

Interannual average values in CAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             184   (267  )
-- River discharge ---                   10.03    (92.31   )
-- Sediment resuspension ---             144.36   (144.36  )
-- Sediment released by sea ice ---      29.94    (29.94   )
-- Dust released by sea ice ---          0.01     (0.01    )
-- Dust deposition ---                   0.06     (0.06    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  5.44  (34.61)
-- Sediment resuspension ---            78.29 (54.13)
-- Sediment released by sea ice ---     16.24 (11.23)
-- Dust released by sea ice ---         0.01  (0.01 )
-- Dust deposition ---                  0.03  (0.02 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    154  (223  )
-- River discharge ---

In [40]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='CAA')

Interannual average values in CAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             507   (657  )
-- River discharge ---                   17.78    (167.80  )
-- Sediment resuspension ---             431.89   (431.89  )
-- Sediment released by sea ice ---      56.66    (56.66   )
-- Dust released by sea ice ---          0.26     (0.26    )
-- Dust deposition ---                   0.02     (0.02    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  3.51  (25.56)
-- Sediment resuspension ---            85.25 (65.77)
-- Sediment released by sea ice ---     11.18 (8.63 )
-- Dust released by sea ice ---         0.05  (0.04 )
-- Dust deposition ---                  0.00  (0.00 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    1666 (1791 )
-- River discharge ---

In [58]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='wCAA')

Interannual average values in wCAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             18    (20   )
-- River discharge ---                   0.56     (2.34    )
-- Sediment resuspension ---             11.42    (11.42   )
-- Sediment released by sea ice ---      5.86     (5.86    )
-- Dust released by sea ice ---          0.00     (0.00    )
-- Dust deposition ---                   0.00     (0.00    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  3.16  (11.92)
-- Sediment resuspension ---            63.97 (58.19)
-- Sediment released by sea ice ---     32.84 (29.87)
-- Dust released by sea ice ---         0.01  (0.01 )
-- Dust deposition ---                  0.02  (0.02 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    35   (39   )
-- River discharge --

In [59]:
print_interannual_averages(final_ref_totals, final_spm_totals, final_ref_averages, final_spm_averages, location='eCAA')

Interannual average values in eCAA
----------------------------------------------------------------
Total annual contribution of Mn [Mmol/yr]:             33    (57   )
-- River discharge ---                   2.48     (25.77   )
-- Sediment resuspension ---             24.08    (24.08   )
-- Sediment released by sea ice ---      6.77     (6.77    )
-- Dust released by sea ice ---          0.00     (0.00    )
-- Dust deposition ---                   0.00     (0.00    )
----------------------------------------------------------------
Total annual contribution of Mn [%]:
-- River discharge ---                  7.45  (45.51)
-- Sediment resuspension ---            72.23 (42.53)
-- Sediment released by sea ice ---     20.30 (11.95)
-- Dust released by sea ice ---         0.01  (0.01 )
-- Dust deposition ---                  0.01  (0.01 )
----------------------------------------------------------------
Average annual contribution of Mn [micromol/m2/yr]:    49   (82   )
-- River discharge --